In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uj_fasci. Use `wandb login --relogin` to force relogin


True

In [3]:
# Prepare the training dataset
BATCH_SIZE = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Build input pipeline
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [4]:
def make_model():
    inputs = keras.Input(shape=(784,), name='digits')
    x1 = keras.layers.Dense(64, activation='relu')(inputs)
    x2 = keras.layers.Dense(64, activation='relu')(x1)
    outputs = keras.layers.Dense(10, name='predictions')(x2)

    return keras.Model(inputs=inputs, outputs=outputs)

In [5]:
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

In [6]:
def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

In [7]:
def train(train_dataset, val_dataset, model, optimizer,
          train_acc_metric, val_acc_metric,
          epochs=10, log_step=200, val_log_step=50):
    
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train,
                                    model, optimizer,
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val,
                                       model, loss_fn,
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))

        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        val_acc_metric.reset_states()

        # log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc),
                   'val_loss': np.mean(val_loss),
                   'val_acc': float(val_acc)})

In [8]:
config = {
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 64,
    "log_step": 200,
    "val_log_step": 50,
    "architecture": "CNN",
    "dataset": "CIFAR-10"
}

run = wandb.init(project='my-tf-integration', config=config)
config = wandb.config

# Initialize model
model = make_model()

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      val_dataset,
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs,
      log_step=config.log_step,
      val_log_step=config.val_log_step)

run.finish()


Start of epoch 0
Training acc over epoch: 0.6992
Validation acc: 0.8165

Start of epoch 1
Training acc over epoch: 0.8382
Validation acc: 0.8571

Start of epoch 2
Training acc over epoch: 0.8716
Validation acc: 0.8811

Start of epoch 3
Training acc over epoch: 0.8895
Validation acc: 0.8922

Start of epoch 4
Training acc over epoch: 0.9002
Validation acc: 0.8998

Start of epoch 5
Training acc over epoch: 0.9085
Validation acc: 0.9067

Start of epoch 6
Training acc over epoch: 0.9125
Validation acc: 0.9084

Start of epoch 7
Training acc over epoch: 0.9179
Validation acc: 0.9146

Start of epoch 8
Training acc over epoch: 0.9208
Validation acc: 0.9159

Start of epoch 9
Training acc over epoch: 0.9250
Validation acc: 0.9199


NameError: name 'run_finish' is not defined